In [1]:
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

In [3]:
def load_model(filename):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    # return torch.load(save_filename, map_location='cpu')
    return torch.load(save_filename)


def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.dat', mode='rb'))

In [4]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        
        embed = self.embedding(nn_input.long())
        lstm_out, hidden = self.lstm(embed, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.fc(lstm_out)
        out = out.view(batch_size, -1, self.output_size)
        out = out[:, -1]

        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        # initialize hidden state with zero weights, and move to GPU if available
        
        return hidden

In [5]:
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    tokens = dict()
    tokens[' <period>'] = '.'
    tokens[' <exclamation_mark>'] = '!'
    tokens[' <question_mark>'] = '?'
    tokens[' <comma>'] = ','
    tokens[' <quotation_mark>'] = '"'
    tokens[' <semicolon>'] = ';'
    tokens[' <left_paren>'] = '('
    tokens[' <right_paren>'] = ')'
    tokens[' <dash>'] = '-'
    tokens[' <new_line>'] = '\n'
    return tokens

def clean_text(text):
    # Iterate through all tokens we want to remove
    tokens = token_lookup()
    for k, v in tokens.items():
        text = text.replace(k, v)

    # Reduce to lower case to reduce complexity of words
    text = text.lower()
    return text

In [7]:
int_text, vocab_to_int, int_to_vocab, token_dict = load_preprocess()

trained_rnn = load_model('./save/trained_rnn_trimmed')

In [8]:
sequence_length = 20
gen_length = 400 # modify the length to your preference
prime_word = 'there' # name for starting the script

pad_word = '<PAD>'
generated_script = generate(trained_rnn, vocab_to_int[prime_word], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)

print(clean_text(generated_script))

there, and then earl sawyer and musides were heard.
 the night, the flowers was warped and deserted.
 there was a breath of fear, but a swift, formless sort of diseased horror.
" iä! iä! iä! yog- sothoth!,
 yog- sothoth",
 answer to the institution of the gods.

" i am iranon, dear sir, and commander" of my.
 my friend, however, have told me of what was the of the funeral of akeley's original family, and that it might have been very closely shaken in the absence of the speaker.
 the doctor had been able to secure him, and was still reckoned to his mind, but the old man himself suggested that he was indeed compelled to be believed talking.

 ii.

 the final horror of the numberless clock was very great and exceedingly regular. it was a terrible of this affair, but it was clearly a queer and perplexing thing in the dark, windowless peeling of ancient myth.
 the hindoo paused at once, and was still weak and sunny.
 it was the shrieking of that final raiding scene- the quasi- human monster

In [9]:
"""
trained_rnn.eval()
    
pad_value = vocab_to_int[pad_word]
prime_id = vocab_to_int[prime_word]
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

# create a sequence (batch_size=1) with the prime_id
current_seq = np.full((1, sequence_length), pad_value)
current_seq[-1][-1] = prime_id
predicted = [int_to_vocab[prime_id]]

if train_on_gpu:
    current_seq = torch.LongTensor(current_seq).cuda()
else:
    current_seq = torch.LongTensor(current_seq)

# current_seq = torch.LongTensor(current_seq).cpu()
# initialize the hidden state
hidden = trained_rnn.init_hidden(current_seq.size(0))

# get the output of the rnn
output, _ = trained_rnn(current_seq, hidden)
"""

'\ntrained_rnn.eval()\n    \npad_value = vocab_to_int[pad_word]\nprime_id = vocab_to_int[prime_word]\nos.environ[\'CUDA_LAUNCH_BLOCKING\'] = "1" \n\n# create a sequence (batch_size=1) with the prime_id\ncurrent_seq = np.full((1, sequence_length), pad_value)\ncurrent_seq[-1][-1] = prime_id\npredicted = [int_to_vocab[prime_id]]\n\nif train_on_gpu:\n    current_seq = torch.LongTensor(current_seq).cuda()\nelse:\n    current_seq = torch.LongTensor(current_seq)\n\n# current_seq = torch.LongTensor(current_seq).cpu()\n# initialize the hidden state\nhidden = trained_rnn.init_hidden(current_seq.size(0))\n\n# get the output of the rnn\noutput, _ = trained_rnn(current_seq, hidden)\n'